# Azure Maps - Air quality

https://azure.microsoft.com/en-us/products/azure-maps/?msockid=2e39c66c693c66a5151fd200687567d0

https://learn.microsoft.com/en-us/azure/azure-maps/

In [1]:
import os
import requests
import json
import sys

from datetime import datetime
from dotenv import load_dotenv
from typing import Dict, List, Optional

In [2]:
sys.version

'3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]'

In [3]:
load_dotenv("azure.env")

subscription_key = os.getenv('AZURE_MAPS_KEY')

## Helper

In [4]:
class AzureMapsAirQuality:
    """
    A comprehensive class to interact with Azure Maps Weather API for Air Quality data.
    
    Demonstrates capabilities of:
    - Get Current Air Quality
    - Get Air Quality Daily Forecasts  
    - Get Air Quality Hourly Forecasts
    """
    
    def __init__(self, subscription_key: str):
        """
        Initialize the Azure Maps Air Quality client.
        
        Args:
            subscription_key (str): Azure Maps subscription key
        """
        self.subscription_key = subscription_key
        self.base_url = "https://atlas.microsoft.com/weather"
        self.api_version = "1.1"
        
    def _make_request(self, endpoint: str, params: Dict) -> Optional[Dict]:
        """
        Make HTTP request to Azure Maps API.
        
        Args:
            endpoint (str): API endpoint
            params (Dict): Query parameters
            
        Returns:
            Optional[Dict]: JSON response or None if error
        """
        params['subscription-key'] = self.subscription_key
        params['api-version'] = self.api_version
        
        try:
            response = requests.get(f"{self.base_url}/{endpoint}", params=params)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"Error making request: {e}")
            return None
    
    def get_current_air_quality(self, lat: float, lon: float, 
                              language: str = "en-US", 
                              pollutants: bool = True) -> Optional[Dict]:
        """
        Get current air quality information for a location.
        
        Args:
            lat (float): Latitude 
            lon (float): Longitude
            language (str): Language for results (default: en-US)
            pollutants (bool): Include detailed pollutant info (default: True)
            
        Returns:
            Optional[Dict]: Current air quality data
        """
        endpoint = "airQuality/current/json"
        params = {
            'query': f"{lat},{lon}",
            'language': language,
            'pollutants': str(pollutants).lower()
        }
        
        return self._make_request(endpoint, params)
    
    def get_air_quality_daily_forecast(self, lat: float, lon: float,
                                     duration: int = 3,
                                     language: str = "en-US",
                                     pollutants: bool = True) -> Optional[Dict]:
        """
        Get daily air quality forecasts (1-7 days).
        
        Args:
            lat (float): Latitude
            lon (float): Longitude  
            duration (int): Number of days (1-7, default: 3)
            language (str): Language for results (default: en-US)
            pollutants (bool): Include detailed pollutant info (default: True)
            
        Returns:
            Optional[Dict]: Daily forecast data
        """
        endpoint = "airQuality/forecasts/daily/json"
        params = {
            'query': f"{lat},{lon}",
            'duration': duration,
            'language': language,
            'pollutants': str(pollutants).lower()
        }
        
        return self._make_request(endpoint, params)

    def analyze_air_quality(self, data: Dict) -> Dict:
        """
        Analyze air quality data and provide insights.
        
        Args:
            data (Dict): Air quality response data
            
        Returns:
            Dict: Analysis results
        """
        if not data or 'results' not in data:
            return {'error': 'Invalid data provided'}
        
        results = data['results']
        analysis = {
            'summary': {},
            'pollutants_analysis': {},
            'recommendations': []
        }
        
        for result in results:
            # Basic summary
            analysis['summary'] = {
                'datetime': result.get('dateTime'),
                'air_quality_index': result.get('index'),
                'global_index': result.get('globalIndex'), 
                'category': result.get('category'),
                'dominant_pollutant': result.get('dominantPollutant'),
                'description': result.get('description')
            }
            
            # Pollutants analysis
            if 'pollutants' in result:
                pollutants = {}
                for pollutant in result['pollutants']:
                    pollutants[pollutant['type']] = {
                        'name': pollutant['name'],
                        'concentration': pollutant['concentration']['value'],
                        'unit': pollutant['concentration']['unit'],
                        'index': pollutant['index'],
                        'global_index': pollutant['globalIndex']
                    }
                analysis['pollutants_analysis'] = pollutants
            
            # Generate recommendations based on category
            category = result.get('category', '').lower()
            if category in ['good', 'excellent']:
                analysis['recommendations'].append("Air quality is good - outdoor activities are recommended")
            elif category in ['fair', 'moderate']:
                analysis['recommendations'].append("Air quality is acceptable - sensitive individuals should limit prolonged outdoor activities")
            elif category in ['poor', 'unhealthy']:
                analysis['recommendations'].append("Air quality is poor - limit outdoor activities, especially for sensitive groups")
            elif category in ['very poor', 'hazardous']:
                analysis['recommendations'].append("Air quality is hazardous - avoid outdoor activities")
        
        return analysis
    

## Examples

### Azure Maps Air Quality API Demo

In [5]:
client = AzureMapsAirQuality(subscription_key)

In [6]:
locations = {
    "Los Angeles, CA": (34.0522, -118.2437),
    "New York, NY": (40.7128, -74.0060),
    "Paris, France": (48.8566, 2.3522),
    "Seattle, WA": (47.632346, -122.13887),
    "Tokyo, Japan": (35.6895, 139.6917)
}

In [7]:
print("🌍 Azure Maps Air Quality API\n")
    
for city, (lat, lon) in locations.items():
    print(f"\n📍 Air Quality Data for {city}")
    print("-" * 120)

    current_data = client.get_current_air_quality(lat, lon)
    if current_data:
        analysis = client.analyze_air_quality(current_data)
          
        print("🔍 Current Air Quality:")
        print(f"  • Category: {analysis['summary'].get('category')}")
        print(f"  • Air Quality Index = {analysis['summary'].get('air_quality_index')}")
        print(f"  • Dominant Pollutant: {analysis['summary'].get('dominant_pollutant')}")
        print(f"  • Description: {analysis['summary'].get('description')}")
            
        if analysis['pollutants_analysis']:
            print("\n💨 Pollutant Concentrations:")
            for pollutant, data in analysis['pollutants_analysis'].items():
                print(f"  • {data['name']}: {data['concentration']} {data['unit']} (Index: {data['index']})")
            
        if analysis['recommendations']:
            print("\n💡 Recommendations:")
            for rec in analysis['recommendations']:
                print(f"  • {rec}")
                print()

🌍 Azure Maps Air Quality API


📍 Air Quality Data for Los Angeles, CA
------------------------------------------------------------------------------------------------------------------------
🔍 Current Air Quality:
  • Category: Poor
  • Air Quality Index = 49.6
  • Dominant Pollutant: Nitrogen Dioxide
  • Description: The air has reached a high level of pollution and is unhealthy for sensitive groups. Reduce time spent outside if you are feeling symptoms such as difficulty breathing or throat irritation.

💨 Pollutant Concentrations:
  • Nitrogen Dioxide: 42.5 µg/m³ (Index: 21.3)
  • Ozone: 18.2 µg/m³ (Index: 7.4)
  • Particulate Matter 10: 26.7 µg/m³ (Index: 24.7)
  • Particulate Matter 2.5: 11.9 µg/m³ (Index: 49.6)
  • Carbon Monoxide: 192.9 µg/m³ (Index: 1.9)
  • Sulfur Dioxide: 8.9 µg/m³ (Index: 4.9)

💡 Recommendations:
  • Air quality is poor - limit outdoor activities, especially for sensitive groups


📍 Air Quality Data for New York, NY
-------------------------------------------

### 3-Day Air Quality Forecast

In [8]:
lat, lon = "47.632346", "-122.13887"

In [9]:
print("3-Day Air Quality Forecast:")

for city, (lat, lon) in locations.items():
    print(f"\nCity = {city}")
    print("-" * 50)
    daily_forecast = client.get_air_quality_daily_forecast(lat, lon, duration=3)
    if daily_forecast and 'results' in daily_forecast:
        for i, day in enumerate(daily_forecast['results'][:3], 1):
            date = datetime.fromisoformat(day['dateTime'].replace('Z', '+00:00'))
            print(f"  Day {i} ({date.strftime('%Y-%m-%d')}): {day.get('category')} - Index = {day.get('index')}")

3-Day Air Quality Forecast:

City = Los Angeles, CA
--------------------------------------------------
  Day 1 (2025-09-01): Poor - Index = 63.9
  Day 2 (2025-09-02): Poor - Index = 54.4
  Day 3 (2025-09-03): Poor - Index = 53.4

City = New York, NY
--------------------------------------------------
  Day 1 (2025-09-01): Poor - Index = 47.4
  Day 2 (2025-09-02): Poor - Index = 43.9
  Day 3 (2025-09-03): Poor - Index = 57.4

City = Paris, France
--------------------------------------------------
  Day 1 (2025-08-31): Fair - Index = 31.4
  Day 2 (2025-09-01): Fair - Index = 28.8
  Day 3 (2025-09-02): Fair - Index = 31.8

City = Seattle, WA
--------------------------------------------------
  Day 1 (2025-09-01): Poor - Index = 42.7
  Day 2 (2025-09-02): Poor - Index = 43.3
  Day 3 (2025-09-03): Poor - Index = 41.8

City = Tokyo, Japan
--------------------------------------------------
  Day 1 (2025-08-31): Poor - Index = 0.0
  Day 2 (2025-09-01): Poor - Index = 0.0
  Day 3 (2025-09-02): P